<a href="https://colab.research.google.com/github/alexalra/Portfolio-Alejandro/blob/main/Python_A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Intro

This project showcases my Python expertise and is part of my portfolio. I utilize Pandas to analyze a marketing dataset and conduct A/B testing. The process involves exploring the dataset, addressing key data analysis questions, modifying the DataFrame, and ultimately performing the A/B test.



In [2]:
#Upload the csv file

from google.colab import files

uploaded = files.upload()


Saving WA_Marketing-Campaign.csv to WA_Marketing-Campaign.csv


In [3]:
#Read the csv file

import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['WA_Marketing-Campaign.csv']))

df.head(5)

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81


### Let's explore the DF to get a sense of the data that it contains

In [ ]:
#Let's explore the DF

df.info()

df.shape

In [ ]:
#Let's have a look at the lasw rows out of curiosity

df.iloc[-10:]

In [ ]:
#Let's see how many different markets there are

df["MarketID"].nunique()

#Let's see which are the different market sizes there are

df["MarketSize"].unique()

#Let's see how many locations there are

df["LocationID"].nunique()

#Let's see how old are the oldest and newest stores

df["AgeOfStore"].max()
df["AgeOfStore"].min()

#Let's see how many promotions were used for this A/B testing programme

df["Promotion"].unique()

#Let's see for how many weeks the experiment was run

df["week"].unique()

#Let's see whether all the promotions were run for the 4 weeks.

promotion_by_week = df.groupby("Promotion")["week"].nunique()

print(promotion_by_week)

Now we know that:


*   The dataframe is composed of 548 rows and 7 columns, with dtypes: float64(1), int64(5), object(1);
*   It comprises data for 10 different markets;
*   There are 3 markets sizes: "Small", "Medium", "Large";
*   The campaign was run in 137 different locations;
*   The oldest store is 28 y/o and the newest 1 y/o;
*   The company implemented 3 different promotions;
*   The campaign was run for 4 weeks;
*   The 3 promotions run for the 4 weeks.










## Data Cleansing and Manipulation

In [4]:
#It turns out that "SalesInThousands" column does not account for taxes.

#The stakeholders want to run the A/B test taking into account  the final net profit.

# We'll add a column with net profit to the DF considering that markets of small size pay 10% in taxes, medium 12% and large 16%.

df["SalesNetInThousands"] = df.apply(lambda row:
                                               row["SalesInThousands"] - row["SalesInThousands"] * 0.10 if row["MarketSize"] == "Small"
                                               else row["SalesInThousands"] - row["SalesInThousands"] * 0.12 if row["MarketSize"] == "Medium"
                                               else row["SalesInThousands"] - row["SalesInThousands"] * 0.16,
                                               axis=1)

df.head(10)


,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands,SalesNetInThousands
0,1,Medium,1,4,3,1,33.73,29.6824
1,1,Medium,1,4,3,2,35.67,31.3896
2,1,Medium,1,4,3,3,29.03,25.5464
3,1,Medium,1,4,3,4,39.25,34.5400
4,1,Medium,2,5,2,1,27.81,24.4728
5,1,Medium,2,5,2,2,34.67,30.5096
6,1,Medium,2,5,2,3,27.98,24.6224
7,1,Medium,2,5,2,4,27.72,24.3936
8,1,Medium,3,12,1,1,44.54,39.1952
9,1,Medium,3,12,1,2,37.94,33.3872


In [23]:
#It appears that columns follow the same convention in terms of formatting. They follow the 'tittle' with the first letter of each words in capitals.

#This is however not the case for 'week'. Let's modify it.

df.rename(columns= {'week': 'Week'}, inplace= True)


,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,Week,SalesInThousands,SalesNetInThousands
0,1,Medium,1,4,3,1,33.73,29.6824
1,1,Medium,1,4,3,2,35.67,31.3896
2,1,Medium,1,4,3,3,29.03,25.5464
3,1,Medium,1,4,3,4,39.25,34.5400
4,1,Medium,2,5,2,1,27.81,24.4728


## Let's dive deeper into the DF

The idea is to obtain an overview of which markets and locations are generating more profits for the company, and whether the market size and age are determining factors.

In [10]:
#Let's see the MarketID with bigger sales in total

Larger_Market = df.groupby(["MarketID", "MarketSize"])["SalesNetInThousands"].sum().reset_index()

Larger_Market.rename(columns={"SalesNetInThousands":"Total_SalesNetInThousands"}, inplace=True)

print(Larger_Market.sort_values(by="Total_SalesNetInThousands", ascending = False).reset_index(drop=True))


   MarketID MarketSize  Total_SalesNetInThousands
0         3      Large                  6281.1084
1        10      Large                  3613.7640
2         5     Medium                  2578.6464
3         7     Medium                  2348.2976
4         8     Medium                  2067.7712
5         6     Medium                  1921.7880
6         9     Medium                  1863.5144
7         4      Small                  1766.0610
8         1     Medium                  1606.2552
9         2      Small                  1334.0430


In [11]:
# Let's see the MarketSize with bigger sales

MarketSize_Sales = df.groupby("MarketSize")["SalesNetInThousands"].sum().reset_index()

# The above gives use the MarketSize with more sales, but doesn't account for the amount of locations in each MarketSize.

# Let's retrieve the sales per market and the amount of locations per each.

MarketSize_Sales_Sales = df.groupby("MarketSize").agg(
    Market_Sales=('SalesNetInThousands', 'sum'),
    Count_Locations=('LocationID', 'nunique')
  ).reset_index()

print(MarketSize_Sales_Sales)


  MarketSize  Market_Sales  Count_Locations
0      Large     9894.8724               42
1     Medium    12386.2728               80
2      Small     3100.1040               15


In [21]:
# Let's see the market locations with bigger sales

Location_Sales = df.groupby(["LocationID", "MarketSize", "AgeOfStore"])["SalesNetInThousands"].sum().reset_index()

Location_Sales.rename(columns= {"SalesNetInThousands": "SalesNetInThousands_PerLocation"}, inplace=True)

print(Location_Sales.sort_values(by= "SalesNetInThousands_PerLocation", ascending = False).reset_index(drop= True))

     LocationID MarketSize  AgeOfStore  SalesNetInThousands_PerLocation
0           209      Large           1                         319.5024
1           218      Large           2                         306.1800
2           220      Large           3                         303.5508
3           210      Large          19                         299.9220
4           208      Large           1                         297.6204
..          ...        ...         ...                              ...
132           2     Medium           5                         103.9984
133         510     Medium           7                         103.7960
134           5     Medium          10                         102.0888
135         507     Medium           5                          99.0616
136           8     Medium          10                          97.9968

[137 rows x 4 columns]


Some results from this first analysis:

*   When looking at data from a Market perspective, large markets are the ones generating more profits, namely 3 and 10 are the most profitable markets.
*   Further, there appears to be a correlation between size and revenue, with medium and small size markets generating less revenue than big ones.
*   It is worth noting that the large majority of markets are of size "Medium".
*   When it comes to the location ID, locations with 200s code lead the way with the highest number in sales.
*   When it comes to the age of the store, it does not appear to be a determining factor as the top 5 stores generating the highest profits are 1, 2 and 3, 19 and 1 years old respectively.





## Let's perform the A/B test




In [42]:
#Let's see the sales per promotion per week and the difference between each week

weekly_sales = df.groupby(["Promotion","Week"])["SalesNetInThousands"].sum().reset_index()

weekly_sales.rename(columns={"Week":"WeekAB"}, inplace= True)

weekly_sales["Weekly_Difference"] = weekly_sales.groupby((weekly_sales["WeekAB"] == 1).cumsum())["SalesNetInThousands"].diff()

#Let's print the results for each promotion

print(weekly_sales[weekly_sales["Promotion"] == 1].sort_values(by = "WeekAB", ascending = False).reset_index(drop = True))

print(weekly_sales[weekly_sales["Promotion"] == 2].sort_values(by = "WeekAB", ascending = False).reset_index(drop = True))

print(weekly_sales[weekly_sales["Promotion"] == 3].sort_values(by = "WeekAB", ascending = False).reset_index(drop = True))

   Promotion  WeekAB  SalesNetInThousands  Weekly_Difference
0          1       4            2174.4650           -13.2188
1          1       3            2187.6838            68.2424
2          1       2            2119.4414           -48.3714
3          1       1            2167.8128                NaN
   Promotion  WeekAB  SalesNetInThousands  Weekly_Difference
0          2       4            1880.4200           -57.6068
1          2       3            1938.0268             3.8496
2          2       2            1934.1772            -5.2122
3          2       1            1939.3894                NaN
   Promotion  WeekAB  SalesNetInThousands  Weekly_Difference
0          3       4            2260.0718            40.9678
1          3       3            2219.1040           -65.8216
2          3       2            2284.9256             9.1942
3          3       1            2275.7314                NaN


In [49]:
#Let's see now simply which Promotion generated higher sales during the 4 weeks

weekly_sales.groupby("Promotion")["SalesNetInThousands"].sum().reset_index()


#Over the course of the 4 weeks, how did the sales of each Promotion increase or decreased?

weekly_sales.groupby("Promotion")["Weekly_Difference"].sum().reset_index()

#How did the sales of all the campaigns fluctuated throughout the 4 weeks?

weekly_sales.groupby("WeekAB")["Weekly_Difference"].sum().reset_index()


,WeekAB,Weekly_Difference
0,1,0.0000
1,2,-44.3894
2,3,6.2704
3,4,-29.8578


## Conclusions from the A/B test

*   Promotion 3 is the one that generated more sales in thousands
*   Nonetheless, over the course of the 4 weeks, only promotion 1 generated more profits by the end of the campaign than at the beginning.
*  Both promotions 2 and 3 registered lower numbers in sales by the end of the
campaign than at the beginning.
*  When considering all campaings, the sales decreased between the first and the second week.
*  Sales increased between the second and the third week.
*  Sales fell once more between the third and the fourth week.


The main takeaway is that, while the 3rd campaign has generated more profits throughout the 4 weeks, it showed a worrysome tendency with drastic fluctuations, and with the 4th week of the campaign, registering lower sales than at the beginning.






